In [50]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import textwrap

from generate import *
from helpers import *

### Import model

In [ ]:
model = torch.load('models/all.txt.model')

### Generate text

In [54]:
prime_str = '\n\n'
text, hiddens = generate(model, prime_str, 500, 0.8, True, output_hiddens=True)
plot_colored_text(text, np.full(len(text), -1))

### Generate big plot with all neurons

In [60]:
def big_plot(text, colors, W, zoom=1.0, cols=4, save_file=None):
    ''' Plot wrapped rnn output with each color representing the activation 
        of a certain node in the hidden representation '''
    N = colors.shape[-1]
    rows = math.ceil(N/cols)
    
    plt.figure(figsize=(cols*zoom*W/5, rows*zoom*len(text)/W/3))
    plt.subplots_adjust(wspace = 0.01, hspace = .01)
    for i in range(N):
        plt.subplot(rows, cols, i+1)
        heat = colors[i].flatten()
        wrapped_text, wrapped_heat = wrap_colored_text(text, heat, W=60)
        sns.heatmap(wrapped_heat, annot=wrapped_text, fmt='s',
                    cbar=False, cmap='coolwarm')
        plt.xticks([])
        plt.yticks([])
        plt.title('Node {}'.format(i))
    if save_file:
        plt.savefig(save_file)
    plt.show()

In [61]:
W = 60
big_plot(text, hiddens.T[:20], W=W, zoom=1.0, save_file='plots/neurons-0-20.png')
big_plot(text, hiddens.T[20:40], W=W, zoom=1.0, save_file='plots/neurons-20-40.png')
big_plot(text, hiddens.T[40:60], W=W, zoom=1.0, save_file='plots/neurons-40-60.png')
big_plot(text, hiddens.T[60:80], W=W, zoom=1.0, save_file='plots/neurons-60-80.png')
big_plot(text, hiddens.T[80:100], W=W, zoom=1.0, save_file='plots/neurons-80-100.png')

IndexError: index 20 is out of bounds for axis 0 with size 20